# Lossy visualization with Figurl

This notebook produces the visualization links of the `Appendix A` in the paper.

We assumes the `ephys-compression/scripts/benchmark-lossly-exp.py` scripts have been run and the  `../data/ephys-compression-results/results-lossy-exp` are available. Specifically, this notebook uses the  `../data/ephys-compression-results/results-lossy-exp/compressed_recordings` folder, which contain short 5-s snippets of the datasets after lossy compression. 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import numpy as np
import os
from pathlib import Path
import pandas as pd
import json

import spikeinterface as si
import spikeinterface.preprocessing as spre
import spikeinterface.widgets as sw

%matplotlib widget

In [ ]:
kachery_client = os.getenv("KACHERY_CLOUD_CLIENT_ID")
kachery_key = os.getenv("KACHERY_CLOUD_PRIVATE_KEY")
kachery_zone = os.getenv("KACHERY_ZONE")

In [ ]:
if not (kachery_client and kachery_key):
    warnings.warn("KACHERY secrets (KACHERY_CLOUD_CLIENT_ID and KACHERY_CLOUD_PRIVATE_KEY) not found. "
                  "Visualization might fail!")
if kachery_zone is None and kachery_zone != "aind":
    warnings.warn("KACHERY_ZONE not set to 'aind'")

In [ ]:
data_folder = Path("../data/")
results_folder = Path("../results/")

In [ ]:
# load lossy results
results_lossy_exp_folder = data_folder / "ephys-compression-results" / "results-lossy-exp"

res_lossy_exp = pd.read_csv(results_lossy_exp_folder / "benchmark-lossy-exp.csv", index_col=False)
compressed_recordings_folder = results_lossy_exp_folder / "compressed_recordings"

In [ ]:
probe_names = {"Neuropixels1.0": "NP1",
               "Neuropixels2.0": "NP2"}
for probe, probe_name in probe_names.items():
    res_lossy_exp.loc[res_lossy_exp.query(f"probe == '{probe}'").index, "probe"] = probe_name
probes = ["NP1", "NP2"]

In [ ]:
strategies = ["wavpack", "bit_truncation"]

factors = {
    "wavpack": [0, 6, 5, 4, 3.5, 3, 2.5, 2.25],
    "bit_truncation": [0, 1, 2, 3, 4, 5, 6, 7]
}

fig_titles = {
    "bit_truncation": "Bit truncation",
    "wavpack": "WavPack Hybrid"   
}

sessions = np.unique(res_lossy_exp.session)

In [ ]:
time_range_viz = np.array([2, 2.5])
clim = (-50, 50)

In [ ]:
viz_links = {}

rec_comps = {}
for session in sessions:
    viz_links[session] = {}

    probe_name = np.unique(res_lossy_exp.query(f"session == '{session}'").probe)[0]
    print(f"Plotting session: {probe_name} - {session}")

    for strategy in strategies:
        print(f"\tStrategy {strategy}")
        rec_comp_dict = {}
        for factor in factors[strategy]:
            rec = si.read_zarr(compressed_recordings_folder / f"{session}_{strategy}-{factor}.zarr")
            rec_proc = spre.highpass_filter(rec)
            rec_proc = spre.common_reference(rec_proc)
            rec_comp_dict[str(factor)] = rec_proc
        try:
            # generate link
            w = sw.plot_timeseries(rec_comp_dict,
                                   time_range=time_range_viz, clim=clim,
                                   return_scaled=True,
                                   figlabel=f"{probe_name} - {session} - {fig_titles[strategy]}",
                                   order_channel_by_depth=True,
                                   backend="sortingview")
            viz_links[session][strategy] = w.url
        except Exception as e:
            print(f"Failed to visualize {session} - {strategy}\n\n{e}")

In [ ]:
# save viz in output
viz_output = json.dumps(viz_links, indent=4)
# replace special characters
viz_output = viz_output.replace('\\"', "%22")
viz_output = viz_output.replace('#', "%23")

# save vizualization output
visualization_output_file = results_folder / "visualization_output.json"
# remove escape characters
visualization_output_file.write_text(viz_output)

# save to latex
lines = []
lines.append("\\begin{itemize}")
for session, viz_dict in viz_links.items():
    session_ltx = session.replace("_", "\\_")
    lines.append("\t\\item " + session_ltx)

    if len(viz_dict) > 0:
        lines.append("\t\t\\begin{itemize}")
        
    # bit truncation
    if "bit_truncation" in viz_dict:
        link = viz_dict["bit_truncation"]
        name = "Bit truncation"
        lines.append("\t\t\t\\item \\href{" + f"{link}" + "}{" + f"{name}" + "}")
    # WVH
    if "wavpack" in viz_dict:
        link = viz_dict["wavpack"]
        name = "\\texttt{WavPack} Hybrid"
        lines.append("\t\t\t\\item \\href{" + f"{link}" + "}{" + f"{name}" + "}")
    if len(viz_dict) > 0:
        lines.append("\t\t\\end{itemize}")
lines.append("\\end{itemize}")

with (results_folder / "vis_latex.txt").open("w") as f:
    for l in lines:
        print(l)
        f.write(f"{l}\n")